In [1]:
import requests

def fetch_cves(limit=10000):
    base_url = "https://access.redhat.com/labs/securitydataapi/cve.json"
    cves = []
    page = 1
    per_page = 100  # Number of CVEs per page (max allowed by the API)

    while len(cves) < limit:
        url = f"{base_url}?page={page}&per_page={per_page}"
        response = requests.get(url)
        
        if response.status_code != 200:
            print(f"Failed to fetch data: {response.status_code}")
            break
        
        data = response.json()
        if not data:
            break  # No more data available
        
        cves.extend(data)
        page += 1

        print(f"Fetched {len(cves)} CVEs so far...")

        if len(data) < per_page:
            break  # No more pages available

    return cves[:limit]

# Fetch the first 20,000 CVEs
cves = fetch_cves(limit=20000)

# Print the total number of CVEs fetched
print(f"Total CVEs fetched: {len(cves)}")

Fetched 100 CVEs so far...
Fetched 200 CVEs so far...
Fetched 300 CVEs so far...
Fetched 400 CVEs so far...
Fetched 500 CVEs so far...
Fetched 600 CVEs so far...
Fetched 700 CVEs so far...
Fetched 800 CVEs so far...
Fetched 900 CVEs so far...
Fetched 1000 CVEs so far...
Fetched 1100 CVEs so far...
Fetched 1200 CVEs so far...
Fetched 1300 CVEs so far...
Fetched 1400 CVEs so far...
Fetched 1500 CVEs so far...
Fetched 1600 CVEs so far...
Fetched 1700 CVEs so far...
Fetched 1800 CVEs so far...
Fetched 1900 CVEs so far...
Fetched 2000 CVEs so far...
Fetched 2100 CVEs so far...
Fetched 2200 CVEs so far...
Fetched 2300 CVEs so far...
Fetched 2400 CVEs so far...
Fetched 2500 CVEs so far...
Fetched 2600 CVEs so far...
Fetched 2700 CVEs so far...
Fetched 2800 CVEs so far...
Fetched 2900 CVEs so far...
Fetched 3000 CVEs so far...
Fetched 3100 CVEs so far...
Fetched 3200 CVEs so far...
Fetched 3300 CVEs so far...
Fetched 3400 CVEs so far...
Fetched 3500 CVEs so far...
Fetched 3600 CVEs so far...
F

In [2]:
cve_resources=[]
for cve in cves:
    cve_resources.append(cve['resource_url'])

In [3]:
len(cve_resources)

20000

In [4]:
from pandas import DataFrame
cve_names = []
package_names = []
fix_states = []

In [5]:
PYXIS_URL="https://catalog.redhat.com/api/containers/v1"

In [6]:
image_tags = []
grades = []

In [7]:
BAD_GRADES = {'C', 'D', 'E', 'F'}

# Cache to store image data for packages
image_data_cache = {}

for cve_resource in cve_resources:
    cve_response = requests.get(cve_resource)
    security_data = cve_response.json()
    
    if 'package_state' not in security_data:
        continue
    
    for package in security_data['package_state']:
        if package['fix_state'] == 'Affected' and "openshift4/" in package['package_name']:
            package_name = package['package_name']
            
            # Check cache first
            if package_name not in image_data_cache:
                endpoint = f"{PYXIS_URL}/repositories/registry/registry.access.redhat.com/repository/{package_name}/images?page_size=100&sort_by=creation_date[desc]"
                r = requests.get(endpoint)
                image_data = r.json()
                image_data_cache[package_name] = image_data['data'] if image_data.get('data') else []
            
            image_data_list = image_data_cache[package_name]
            
            for data in image_data_list:
                if 'freshness_grades' not in data:
                    continue
                
                for grade in data['freshness_grades']:
                    if grade['grade'] in BAD_GRADES:
                        cve_names.append(security_data['name'])
                        package_names.append(package_name)
                        fix_states.append(package['fix_state'])
                        image_tags.append(data['image_id'])
                        grades.append(grade['grade'])
                        break  # Exit the inner loop once a bad grade is found
                else:
                    continue  # Continue to the next data if no bad grade is found
                break  # Exit the outer loop once a bad grade is found


In [8]:
len(cve_names) , len(package_names) , len(fix_states) , len(grades), len(image_tags)

(664, 664, 664, 664, 664)

In [9]:
# !pip install openpyxl

In [10]:
# security_data['name']+','+package['package_name']+ ','+package['fix_state'] + "\n"
df = DataFrame({'CVE': cve_names, 'package': package_names, 'Fix State': fix_states, 'tag': image_tags, 'grade': grades})
df.to_excel('affected_catchall.xlsx', sheet_name='sheet1', index=False)